<a href="https://colab.research.google.com/github/Ronaa-MX/DNA_Barcoding_PumaBurros/blob/main/Modelo_RedNeuronalArtificial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Red Neuronal Artificial para clasificación de especies a partir de secuencias de ADN
---
**Realizado por : Aarón Ortiz Mendoza**

**Test : Barriba Vega Maria Fernanda**

**Equipo: Puma Burros**



In [ ]:
!python --version

Python 3.10.12


In [ ]:
import sklearn
sklearn.__version__

'1.2.2'

Proceso para conectar **Google Colab** con **Google Sheets**:


```python
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
```



Proceso para conectar **Google Colab** con **Google Drive**:


```python
from google.colab import drive
drive.mount('/content/drive')
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importación de líbrerias y el dataset
---


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#En la URL en vez de poner proyectos/Macroentrenamiento, pueden saltar directamente a PumaBurros-Trabajo
#La URL que los deberían usar es /content/drive/MyDrive/PumaBurros-Trabajo/Datasets/Drosophila_train.csv
path_dataset='/content/drive/MyDrive/Drosophila_train.csv'
seq_mosca_train = pd.read_csv(path_dataset, sep=';')
seq_mosca_train.shape

(504, 667)

In [ ]:
seq_mosca_train

,ID,SPP,S1,S2,S3,S4,S5,S6,S7,S8,...,S656,S657,S658,S659,S660,S661,S662,S663,Unnamed: 665,Unnamed: 666
0,DQ471538_110189402,Drosophila_angor,A,T,T,G,G,A,A,C,...,A,T,T,T,A,G,T,T,NaN,NaN
1,DQ471539_110189404,Drosophila_angor,A,T,C,G,G,A,A,C,...,A,T,T,T,A,T,T,G,NaN,NaN
2,DQ471540_110189406,Drosophila_angor,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
3,DQ471541_110189408,Drosophila_angor,A,T,C,G,G,T,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
4,DQ471542_110189410,Drosophila_angor,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Auxiliar,Auxiliar,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,NaN,NaN
500,Auxiliar,Auxiliar,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,NaN,NaN
501,Auxiliar,Auxiliar,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,NaN,NaN
502,Auxiliar,Auxiliar,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,NaN,NaN


## Limpieza del dataset para análisis de datos
---

In [ ]:
# Dataset con las especies únicas
seq_mosca_noduplicate = seq_mosca_train.drop_duplicates(subset='SPP')
seq_mosca_noduplicate

,ID,SPP,S1,S2,S3,S4,S5,S6,S7,S8,...,S656,S657,S658,S659,S660,S661,S662,S663,Unnamed: 665,Unnamed: 666
0,DQ471538_110189402,Drosophila_angor,A,T,T,G,G,A,A,C,...,A,T,T,T,A,G,T,T,NaN,NaN
10,DQ383668_87475135,Drosophila_arizonae,N,N,N,N,N,N,A,C,...,N,N,N,N,N,N,N,N,NaN,NaN
24,DQ471546_110189418,Drosophila_barutani,A,T,T,G,G,A,A,C,...,A,C,T,T,A,T,T,T,NaN,NaN
29,DQ471557_110189440,Drosophila_beppui,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
32,DQ471544_110189414,Drosophila_daruma,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
36,AY541122_46948544,Drosophila_falleni,A,T,C,G,G,A,A,C,...,A,T,T,T,A,T,T,C,NaN,NaN
48,AY541185_46948670,Drosophila_innubila,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
73,AF200830_8573401,Drosophila_mauritiana,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
76,AJ400907_7799401,Drosophila_melanogaster,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,NaN,NaN,NaN
84,AY533789_42601681,Drosophila_mettleri,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,NaN,NaN


In [ ]:
#Dimensiones del dataset de especies únicas
seq_mosca_noduplicate.shape

(20, 667)

In [ ]:
#Dataset de los ID de especies únicas
seq_mosca_ID=seq_mosca_train[['ID','SPP']]
seq_mosca_ID=seq_mosca_ID.drop_duplicates(subset='ID')
seq_mosca_ID=seq_mosca_ID.drop(labels=499,axis=0)
seq_mosca_ID

,ID,SPP
0,DQ471538_110189402,Drosophila_angor
1,DQ471539_110189404,Drosophila_angor
2,DQ471540_110189406,Drosophila_angor
3,DQ471541_110189408,Drosophila_angor
4,DQ471542_110189410,Drosophila_angor
...,...,...
494,DQ426805_90018989,Drosophila_virilis
495,DQ426806_90018991,Drosophila_virilis
496,DQ426807_90018993,Drosophila_virilis
497,DQ471535_110189396,Drosophila_virilis


In [ ]:
#Dataset sin ID,columnas sin valores y fila auxiliar
seq_mosca_noID=seq_mosca_noduplicate.drop('ID',axis=1)
seq_mosca_noID=seq_mosca_noID.iloc[:, :-2]
seq_mosca_noID=seq_mosca_noID.iloc[:-1, :]
seq_mosca_noID.loc[seq_mosca_noID['SPP'] == 'Drosophila_melanogaster', 'S663'] = 'N'
seq_mosca_noID

,SPP,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S654,S655,S656,S657,S658,S659,S660,S661,S662,S663
0,Drosophila_angor,A,T,T,G,G,A,A,C,T,...,A,C,A,T,T,T,A,G,T,T
10,Drosophila_arizonae,N,N,N,N,N,N,A,C,T,...,N,N,N,N,N,N,N,N,N,N
24,Drosophila_barutani,A,T,T,G,G,A,A,C,T,...,A,C,A,C,T,T,A,T,T,T
29,Drosophila_beppui,A,T,T,G,G,A,A,C,T,...,A,C,A,T,T,T,A,T,T,T
32,Drosophila_daruma,A,T,T,G,G,A,A,C,T,...,A,C,A,T,T,T,A,T,T,T
36,Drosophila_falleni,A,T,C,G,G,A,A,C,T,...,A,C,A,T,T,T,A,T,T,C
48,Drosophila_innubila,A,T,T,G,G,A,A,C,T,...,A,C,A,T,T,T,A,T,T,T
73,Drosophila_mauritiana,A,T,T,G,G,A,A,C,T,...,A,C,A,T,T,T,A,T,T,T
76,Drosophila_melanogaster,A,T,T,G,G,A,A,C,T,...,A,C,A,T,T,T,A,T,T,N
84,Drosophila_mettleri,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [ ]:
#Dataset de cadenas de ADN, especies únicas
columnas_s = [f'S{i}' for i in range(1, 664)]
seq_mosca_noID['Secuencia'] = seq_mosca_noID[columnas_s].apply(lambda x: ''.join(x), axis=1)
seq_adn_moscanoduplicate=seq_mosca_noID[['SPP','Secuencia']]
seq_adn_moscanoduplicate

,SPP,Secuencia
0,Drosophila_angor,ATTGGAACTTTATATTTTATTTTTGGGGCATGGGCTGGTATAGTCG...
10,Drosophila_arizonae,NNNNNNACTTTATATTTTATTTTCGGAGCTTGAGCTGGAATAGCGG...
24,Drosophila_barutani,ATTGGAACTTTATATTTTATTTTCGGAGCATGAGCAGGAATAGTAG...
29,Drosophila_beppui,ATTGGAACTTTATATTTTATTTTCGGAGCATGAGCTGGAATAGTGG...
32,Drosophila_daruma,ATTGGAACTTTATATTTTATTTTTGGTGCATGAGCAGGAATAGTGG...
36,Drosophila_falleni,ATCGGAACTTTATATTTTATTTTTGGTGCTTGAGCCGGAATAGTAG...
48,Drosophila_innubila,ATTGGAACTTTATACTTTATTTTTGGTGCTTGAGCCGGAATAGTAG...
73,Drosophila_mauritiana,ATTGGAACTTTATATTTTATCTTTGGAGCTTGAGCTGGGATAGTAG...
76,Drosophila_melanogaster,ATTGGAACTTTATATTTTATTTTTGGAGCTTGAGCTGGAATAGTTG...
84,Drosophila_mettleri,NNNNNNNNNTTATATTTTATTTTTGGAGCTTGAGCTGGAATAGTTG...


In [ ]:
#Tipos de datos del dataset original
seq_mosca_train.dtypes

ID              object
SPP             object
S1              object
S2              object
S3              object
                 ...  
S661            object
S662            object
S663            object
Unnamed: 665    object
Unnamed: 666    object
Length: 667, dtype: object

In [ ]:
#Tipos de datos del dataset sin ID
seq_mosca_noID.dtypes

SPP          object
S1           object
S2           object
S3           object
S4           object
              ...  
S660         object
S661         object
S662         object
S663         object
Secuencia    object
Length: 665, dtype: object

In [ ]:
#Número total de apariciones de especies en el dataset sin el auxiliar
seq_mosca_ID['SPP'].value_counts()

Drosophila_subquinaria       109
Drosophila_recens            109
Drosophila_pachea             63
Drosophila_mojavensis         38
Drosophila_montana            34
Drosophila_innubila           25
Drosophila_simulans           22
Drosophila_mettleri           19
Drosophila_arizonae           14
Drosophila_falleni            12
Drosophila_angor              10
Drosophila_virilis             9
Drosophila_melanogaster        8
Drosophila_nigrospiracula      8
Drosophila_barutani            5
Drosophila_navojoa             4
Drosophila_daruma              4
Drosophila_mauritiana          3
Drosophila_beppui              3
Name: SPP, dtype: int64

In [ ]:
#Valores únicos de especies del dataset original
seq_mosca_train['SPP'].unique()

array(['Drosophila_angor', 'Drosophila_arizonae', 'Drosophila_barutani',
       'Drosophila_beppui', 'Drosophila_daruma', 'Drosophila_falleni',
       'Drosophila_innubila', 'Drosophila_mauritiana',
       'Drosophila_melanogaster', 'Drosophila_mettleri',
       'Drosophila_mojavensis', 'Drosophila_montana',
       'Drosophila_navojoa', 'Drosophila_nigrospiracula',
       'Drosophila_pachea', 'Drosophila_recens', 'Drosophila_simulans',
       'Drosophila_subquinaria', 'Drosophila_virilis', 'Auxiliar'],
      dtype=object)

In [ ]:
#Valores únicos de las secuencias de ADN de cada especie sin ID
seq_mosca_noID['Secuencia'].unique()

array(['ATTGGAACTTTATATTTTATTTTTGGGGCATGGGCTGGTATAGTCGGTACATCTTTAAGAATTTTAATCCGAGCTGAATTAGGGCACCCAGGAGCTTTAATTGGAGATGACCAAATTTATAACGTAATTGTAACTGCTCATGCTTTTATCATAATTTTTTTTATAGTAATACCAATTATAATTGGAGGATTTGGAAATTGATTAGTCCCTTTAATATTAGGAGCTCCTGATATAGCTTTCCCTCGTATAAATAATATAAGTTTTTGATTATTGCCTCCAGCTTTAACTCTTTTATTAGTAAGAAGTATAGTAGAAAATGGAGCTGGAACAGGATGAACTGTTTACCCTCCTTTATCTGCTGGAATTGCTCATGGGGGTGCTTCAGTAGATCTGGCTATTTTCTCTTTACATTTAGCTGGAATTTCGTCAATTTTAGGAGCAGTAAATTTTATCACAACAGTGATTAATATACGTTCAACTGGTATTACTTTAGACCGAATACCTTTATTTGTATGATCAGTAGTAATTACTGCATTACTTCTGCTTCTATCTTTACCTGTATTGGCAGGAGCAATTACTATACTTTTAACAGACCGAAATTTAAATACATCTTTTTTCGACCCCGCCGGAGGAGGAGACCCAATTTTATACCAACATTTAGTT',
       'NNNNNNACTTTATATTTTATTTTCGGAGCTTGAGCTGGAATAGCGGGAACTTCTTTAAGAATTTTAATTCGTGCCGAATTAGGTCATCCAGGTGCACTAATTGGAGATGATCAAATTTATAATGTAATTGTTACAGCACATGCTTTTGTAATAATTTTTTTTATAGTAATGCCTATTATAATTGGAGGATTTGGAAATTGACTGGTGCCTTTAATACTAGGGGCCCCTGATATAGCATTCCCTCGAATAAATAATNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

In [ ]:
#Número de apariciones totales de las especies del dataset original
seq_mosca_train['SPP'].value_counts()

Drosophila_subquinaria       109
Drosophila_recens            109
Drosophila_pachea             63
Drosophila_mojavensis         38
Drosophila_montana            34
Drosophila_innubila           25
Drosophila_simulans           22
Drosophila_mettleri           19
Drosophila_arizonae           14
Drosophila_falleni            12
Drosophila_angor              10
Drosophila_virilis             9
Drosophila_melanogaster        8
Drosophila_nigrospiracula      8
Drosophila_barutani            5
Auxiliar                       5
Drosophila_navojoa             4
Drosophila_daruma              4
Drosophila_mauritiana          3
Drosophila_beppui              3
Name: SPP, dtype: int64

## Nuevos Datasets en base al ID únicos, especies únicas y sequencias de ADN concatenado
---


In [ ]:
seq_mosca_ID.to_csv('Dropsophilia_ID-Especie_Unicos.csv',sep=';', index=False)
seq_mosca_noID.to_csv('Dropsophilia_Especies-ADN_Unicos.csv',sep=';', index=False)
seq_adn_moscanoduplicate.to_csv('Dropsophilia_Especie-SeqADN_Unicos.csv',sep=';', index=False)

## Pre-procesamiento y separación de los datos en entrenamiento, test y validación
---

In [ ]:
#Pre-preocesamiento de los datos a través de la codificación label y OneHot
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
#Variables de codificación de los datos
label_endoder= LabelEncoder()

In [ ]:
seq_mosca_train

,ID,SPP,S1,S2,S3,S4,S5,S6,S7,S8,...,S656,S657,S658,S659,S660,S661,S662,S663,Unnamed: 665,Unnamed: 666
0,DQ471538_110189402,Drosophila_angor,A,T,T,G,G,A,A,C,...,A,T,T,T,A,G,T,T,NaN,NaN
1,DQ471539_110189404,Drosophila_angor,A,T,C,G,G,A,A,C,...,A,T,T,T,A,T,T,G,NaN,NaN
2,DQ471540_110189406,Drosophila_angor,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
3,DQ471541_110189408,Drosophila_angor,A,T,C,G,G,T,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
4,DQ471542_110189410,Drosophila_angor,A,T,T,G,G,A,A,C,...,A,T,T,T,A,T,T,T,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Auxiliar,Auxiliar,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,NaN,NaN
500,Auxiliar,Auxiliar,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,NaN,NaN
501,Auxiliar,Auxiliar,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,NaN,NaN
502,Auxiliar,Auxiliar,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,NaN,NaN


In [ ]:
#Variable de objetivo del Modelo
Y_train= seq_mosca_train['SPP']
Y_train

0      Drosophila_angor
1      Drosophila_angor
2      Drosophila_angor
3      Drosophila_angor
4      Drosophila_angor
             ...       
499            Auxiliar
500            Auxiliar
501            Auxiliar
502            Auxiliar
503            Auxiliar
Name: SPP, Length: 504, dtype: object

In [ ]:
#Variables a entrenar del modelo
X_train=seq_mosca_train.loc[:,'S1':'S663']
X_train

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S654,S655,S656,S657,S658,S659,S660,S661,S662,S663
0,A,T,T,G,G,A,A,C,T,T,...,A,C,A,T,T,T,A,G,T,T
1,A,T,C,G,G,A,A,C,T,T,...,A,C,A,T,T,T,A,T,T,G
2,A,T,T,G,G,A,A,C,T,T,...,A,C,A,T,T,T,A,T,T,T
3,A,T,C,G,G,T,A,C,A,C,...,A,C,A,T,T,T,A,T,T,T
4,A,T,T,G,G,A,A,C,T,T,...,A,C,A,T,T,T,A,T,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
500,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
501,G,G,G,G,G,G,G,G,G,G,...,G,G,G,G,G,G,G,G,G,G
502,T,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,T,T,T,T,T


In [ ]:
#Separación de la variable a tratar
seq_mosca=seq_mosca_train.loc[:,'SPP':'S663']
seq_mosca.columns

Index(['SPP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9',
       ...
       'S654', 'S655', 'S656', 'S657', 'S658', 'S659', 'S660', 'S661', 'S662',
       'S663'],
      dtype='object', length=664)

In [ ]:
#Codificación One Hot
seq_mosca_X=seq_mosca.loc[:, 'S1':'S663']
seq_mosca_ohe=pd.get_dummies(seq_mosca, columns = seq_mosca_X.columns)
seq_mosca_ohe

,SPP,S1_A,S1_C,S1_G,S1_N,S1_T,S2_A,S2_C,S2_G,S2_N,...,S662_A,S662_C,S662_G,S662_N,S662_T,S663_A,S663_C,S663_G,S663_N,S663_T
0,Drosophila_angor,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,Drosophila_angor,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,Drosophila_angor,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,Drosophila_angor,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,Drosophila_angor,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Auxiliar,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
500,Auxiliar,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
501,Auxiliar,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
502,Auxiliar,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
#Codificación de la variable objectivo
label_endoder.fit(seq_mosca_ohe['SPP'])
seq_mosca_ohe['SPP'] = label_endoder.transform(seq_mosca_ohe['SPP'])
seq_mosca_ohe

,SPP,S1_A,S1_C,S1_G,S1_N,S1_T,S2_A,S2_C,S2_G,S2_N,...,S662_A,S662_C,S662_G,S662_N,S662_T,S663_A,S663_C,S663_G,S663_N,S663_T
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
500,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
501,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
502,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
'''
Antes de trabajar con el dataset quitaremos algunos elementos del data set original para evitar
cierto ruido por parte de eso valores como:
1-Las últimas cinco filas
2-Las últimas dos columnas que no cuentan con valores
3-Cambiar la penúltima fila de una especie dado que no cuenta con un valor, a 'N'
'''
#seq_mosca_train=seq_mosca_train.iloc[:, :-2]
seq_mosca_ohe=seq_mosca_ohe.iloc[:-5, :]
#seq_mosca_train.loc[seq_mosca_train['SPP'] == 'Drosophila_melanogaster', 'S663'] = 'N'
seq_mosca_ohe.shape

(499, 3316)

In [ ]:
list(label_endoder.classes_)

['Auxiliar',
 'Drosophila_angor',
 'Drosophila_arizonae',
 'Drosophila_barutani',
 'Drosophila_beppui',
 'Drosophila_daruma',
 'Drosophila_falleni',
 'Drosophila_innubila',
 'Drosophila_mauritiana',
 'Drosophila_melanogaster',
 'Drosophila_mettleri',
 'Drosophila_mojavensis',
 'Drosophila_montana',
 'Drosophila_navojoa',
 'Drosophila_nigrospiracula',
 'Drosophila_pachea',
 'Drosophila_recens',
 'Drosophila_simulans',
 'Drosophila_subquinaria',
 'Drosophila_virilis']

In [ ]:
#Transformación inversa del dataset codificado a sus especies originales
label_endoder.inverse_transform(seq_mosca_ohe['SPP'])

array(['Drosophila_angor', 'Drosophila_angor', 'Drosophila_angor',
       'Drosophila_angor', 'Drosophila_angor', 'Drosophila_angor',
       'Drosophila_angor', 'Drosophila_angor', 'Drosophila_angor',
       'Drosophila_angor', 'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_barutani', 'Drosophila_barutani',
       'Drosophila_barutani', 'Drosophila_barutani',
       'Drosophila_barutani', 'Drosophila_beppui', 'Drosophila_beppui',
       'Drosophila_beppui', 'Drosophila_daruma', 'Drosophila_daruma',
       'Drosophila_daruma', 'Drosophila_daruma', 'Drosophila_falleni',
       'Drosophila_falleni', 'Drosophila_falleni', 'Drosophila_falleni',
       'Drosophi

In [ ]:
#Separación del dataset codificado a dataset de entrenamiento, prueba y validación
target = seq_mosca_ohe['SPP']
seq_mosca_ohe_X=seq_mosca_ohe.loc[:,'S1_A':'S663_T']
X_train,X_test, Y_train, Y_test=train_test_split(seq_mosca_ohe_X, target, test_size = 0.3, random_state =5, stratify=target)

In [ ]:
#Dataset de entrenamiento y pruebas
print('Training Dataset', X_train.shape)
print('Testing Dataset', X_test.shape)

Training Dataset (349, 3315)
Testing Dataset (150, 3315)


# Modelo de Red Neuronal de Clasificación
---

In [ ]:
# Importación del módulo del modelo a usar
from sklearn.neural_network import MLPClassifier
#Variable del modelo
clf = MLPClassifier(max_iter=300)
clf.fit(X_train,Y_train)
clf.score(X_test,Y_test)

0.9933333333333333

## Métricas del Modelo
---


In [ ]:
#Importación de módulos de las métricas
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, cohen_kappa_score

In [ ]:
#Métricas generales
Y_pred = clf.predict(X_test)
report = classification_report(Y_test,Y_pred)
print(report)
print(Y_pred)

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         8
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         6
          11       1.00      1.00      1.00        11
          12       1.00      1.00      1.00        10
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00        19
          16       0.97      1.00      0.99        33
          17       1.00      1.00      1.00         7
          18       1.00    

In [ ]:
#Métrica estándar
cohen_kappa_score(Y_test,Y_pred)

0.9923187218353134

In [ ]:
#Predicciones del modelo
label_endoder.inverse_transform(Y_pred)

array(['Drosophila_navojoa', 'Drosophila_montana', 'Drosophila_pachea',
       'Drosophila_recens', 'Drosophila_subquinaria', 'Drosophila_recens',
       'Drosophila_recens', 'Drosophila_innubila', 'Drosophila_recens',
       'Drosophila_virilis', 'Drosophila_subquinaria',
       'Drosophila_recens', 'Drosophila_subquinaria', 'Drosophila_recens',
       'Drosophila_montana', 'Drosophila_mojavensis',
       'Drosophila_falleni', 'Drosophila_subquinaria',
       'Drosophila_recens', 'Drosophila_mojavensis',
       'Drosophila_subquinaria', 'Drosophila_subquinaria',
       'Drosophila_subquinaria', 'Drosophila_mojavensis',
       'Drosophila_mettleri', 'Drosophila_recens', 'Drosophila_recens',
       'Drosophila_pachea', 'Drosophila_pachea', 'Drosophila_montana',
       'Drosophila_subquinaria', 'Drosophila_montana',
       'Drosophila_subquinaria', 'Drosophila_simulans',
       'Drosophila_pachea', 'Drosophila_subquinaria', 'Drosophila_daruma',
       'Drosophila_simulans', 'Drosophila_m

In [ ]:
#Predicción con el dataset de test
label_endoder.inverse_transform(Y_test)

array(['Drosophila_navojoa', 'Drosophila_montana', 'Drosophila_pachea',
       'Drosophila_recens', 'Drosophila_subquinaria', 'Drosophila_recens',
       'Drosophila_recens', 'Drosophila_innubila', 'Drosophila_recens',
       'Drosophila_virilis', 'Drosophila_subquinaria',
       'Drosophila_recens', 'Drosophila_subquinaria', 'Drosophila_recens',
       'Drosophila_montana', 'Drosophila_mojavensis',
       'Drosophila_falleni', 'Drosophila_subquinaria',
       'Drosophila_recens', 'Drosophila_mojavensis',
       'Drosophila_subquinaria', 'Drosophila_subquinaria',
       'Drosophila_subquinaria', 'Drosophila_mojavensis',
       'Drosophila_mettleri', 'Drosophila_recens', 'Drosophila_recens',
       'Drosophila_pachea', 'Drosophila_pachea', 'Drosophila_montana',
       'Drosophila_subquinaria', 'Drosophila_montana',
       'Drosophila_subquinaria', 'Drosophila_simulans',
       'Drosophila_pachea', 'Drosophila_subquinaria', 'Drosophila_daruma',
       'Drosophila_simulans', 'Drosophila_m

In [ ]:
#Dataset de predicciones
especie_pred=pd.DataFrame()
especie_pred['Esperado'] = label_endoder.inverse_transform(Y_pred)
especie_pred['Obtenido'] = label_endoder.inverse_transform(Y_test)
especie_pred

,Esperado,Obtenido
0,Drosophila_navojoa,Drosophila_navojoa
1,Drosophila_montana,Drosophila_montana
2,Drosophila_pachea,Drosophila_pachea
3,Drosophila_recens,Drosophila_recens
4,Drosophila_subquinaria,Drosophila_subquinaria
...,...,...
145,Drosophila_mojavensis,Drosophila_mojavensis
146,Drosophila_recens,Drosophila_recens
147,Drosophila_simulans,Drosophila_simulans
148,Drosophila_mojavensis,Drosophila_mojavensis


# Pruebas
---

In [ ]:
#autenticating to google
from google.colab import auth
import gspread
from google.auth import default
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
Drosophila_test = gc.open('Drosophila_test').sheet1
rows_test = Drosophila_test.get_all_values()
df_test = pd.DataFrame(rows_test)
df_test.columns = df_test.iloc[0]
df_test = df_test.iloc[1:]
print(df_test)

0                    ID S1 S2 S3 S4 S5 S6 S7 S8 S9  ... S654 S655 S656 S657  \
1    DQ471543_110189412  A  A  A  T  T  G  G  A  A  ...    C    A    A    C   
2    DQ471554_110189434  A  T  T  G  G  A  A  C  T  ...    A    C    A    T   
3     DQ383671_87475141  N  N  N  N  N  N  A  C  T  ...    N    N    N    N   
4     DQ383677_87475153  N  N  N  N  N  N  A  C  T  ...    N    N    N    N   
5     DQ383678_87475155  N  N  N  N  N  N  A  C  T  ...    N    N    N    N   
..                  ... .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...   
117            Auxiliar  A  A  A  A  A  A  A  A  A  ...    A    A    A    A   
118            Auxiliar  C  C  C  C  C  C  C  C  C  ...    C    C    C    C   
119            Auxiliar  G  G  G  G  G  G  G  G  G  ...    G    G    G    G   
120            Auxiliar  T  T  T  T  T  T  T  T  T  ...    T    T    T    T   
121            Auxiliar  N  N  N  N  N  N  N  N  N  ...    N    N    N    N   

0   S658 S659 S660 S661 S662 S663  
1      A    T  

In [ ]:
df_test.shape

(121, 664)

In [ ]:
#Codificación OneHot del dataset de pruebas
df_X_test = df_test.loc[:, 'S1':'S663']
df_X_test.columns

Index(['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10',
       ...
       'S654', 'S655', 'S656', 'S657', 'S658', 'S659', 'S660', 'S661', 'S662',
       'S663'],
      dtype='object', name=0, length=663)

In [ ]:
#Codificación del dataset de pruebas
one_hot_encoded_test = pd.get_dummies(df_X_test, columns = df_X_test.columns)
print(one_hot_encoded_test)

     S1_A  S1_C  S1_G  S1_N  S1_T  S2_A  S2_C  S2_G  S2_N  S2_T  ...  S662_A  \
1       1     0     0     0     0     1     0     0     0     0  ...       1   
2       1     0     0     0     0     0     0     0     0     1  ...       0   
3       0     0     0     1     0     0     0     0     1     0  ...       0   
4       0     0     0     1     0     0     0     0     1     0  ...       0   
5       0     0     0     1     0     0     0     0     1     0  ...       0   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...     ...   
117     1     0     0     0     0     1     0     0     0     0  ...       1   
118     0     1     0     0     0     0     1     0     0     0  ...       0   
119     0     0     1     0     0     0     0     1     0     0  ...       0   
120     0     0     0     0     1     0     0     0     0     1  ...       0   
121     0     0     0     1     0     0     0     0     1     0  ...       0   

     S662_C  S662_G  S662_N  S662_T  S6

In [ ]:
#Eliminación de las filas con valores auxiliares del dataset codificado
one_hot_encoded_test= one_hot_encoded_test.iloc[:-5, :]
one_hot_encoded_test.shape

(116, 3315)

In [ ]:
X_test.columns

Index(['S1_A', 'S1_C', 'S1_G', 'S1_N', 'S1_T', 'S2_A', 'S2_C', 'S2_G', 'S2_N',
       'S2_T',
       ...
       'S662_A', 'S662_C', 'S662_G', 'S662_N', 'S662_T', 'S663_A', 'S663_C',
       'S663_G', 'S663_N', 'S663_T'],
      dtype='object', length=3315)

In [ ]:
#Predicción a través del modelo
one_hot_encoded_test = one_hot_encoded_test[X_test.columns]
pred = clf.predict(one_hot_encoded_test)
label_endoder.inverse_transform(pred)

array(['Drosophila_angor', 'Drosophila_angor', 'Drosophila_arizonae',
       'Drosophila_arizonae', 'Drosophila_arizonae',
       'Drosophila_barutani', 'Drosophila_falleni', 'Drosophila_falleni',
       'Drosophila_falleni', 'Drosophila_innubila', 'Drosophila_innubila',
       'Drosophila_innubila', 'Drosophila_innubila',
       'Drosophila_melanogaster', 'Drosophila_melanogaster',
       'Drosophila_mettleri', 'Drosophila_mettleri',
       'Drosophila_mettleri', 'Drosophila_mettleri',
       'Drosophila_mettleri', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_mojavensis', 'Drosophila_mojavensis',
       'Drosophila_montana', 'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_montana', 'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_montana', 'Drosophila_montana',
       'Drosophila_nigrospir

In [ ]:
pred.shape

(116,)

In [ ]:
df_test.shape

(121, 664)

In [ ]:
#Eliminación de las filas con valores auxiliares del dataset de pruebas
df_test = df_test.iloc[:-5, :]

In [ ]:
#Dataset predicciones del dataset de test
df_test['pred_spp'] = label_endoder.inverse_transform(pred)
df_test['pred_label'] = pred
print(df_test[['ID','pred_spp','pred_label']])

0                    ID                pred_spp  pred_label
1    DQ471543_110189412        Drosophila_angor           1
2    DQ471554_110189434        Drosophila_angor           1
3     DQ383671_87475141     Drosophila_arizonae           2
4     DQ383677_87475153     Drosophila_arizonae           2
5     DQ383678_87475155     Drosophila_arizonae           2
..                  ...                     ...         ...
112  DQ851680_114187161  Drosophila_subquinaria          18
113  DQ851686_114187173  Drosophila_subquinaria          18
114  DQ851689_114187179  Drosophila_subquinaria          18
115   DQ426800_90018979      Drosophila_virilis          19
116   DQ426803_90018985      Drosophila_virilis          19

[116 rows x 3 columns]


In [ ]:
#Dataset de validación
from gspread_dataframe import set_with_dataframe
title = 'ANN_Pred'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, df_test[['ID','pred_spp','pred_label']])
# include_index=False, include_column_header=True, resize=False